<h1 align="center">Hy  Lisp in Python</h1>
<br>

A quick tutorial for the Hy language. It draws inspiration from [this][HyTut] Hy Notebook which in turns implements much of the Hy tutorial.

I am not sure that I am as enthusiastic as the author of the `Calysto_hi` Jupyter Notebook who is bold enough to claim that Hy is:

* A Lisp that feels very pythonic
* For lispers, a great way to use lisp’s crazy powers but in the wide world of Python’s  libraries (why yes, you now can write a Django application in lisp!)
* For pythonistas, a great way to start exploring lisp, from the comfort of python!
* For everyone: a pleasant language that has a lot of neat ideas!

Instead, I think Hy is really Python with a Lisp flavor **and** some really nice features like macros

## Getting Started

We begin with the obligatory "Hello, World!" and then move to show how one would do X, Y, and Z in Hy

## Hello World in Hy

No surprises here... It's what you hope it would be:

[HyTut]: https://github.com/Calysto/calysto_hy/blob/master/notebooks/Tutorial.ipynb

In [1]:
(print "hello from Hy!")

hello from Hy!


We can do all the usual things, like add numbers, create lexical scopes, define functions, etc. Note that we can use the `+` operator much like one does in Lisp.

In [2]:
(print (+ 1 2 3 4 5 6 7 8 9 10))

(let [result (+ 1 (/ 1 (+ 1 2 3)))]
  result)

55


1.1666666666666667

Hy extends Python to add some data types Lispers might expect, such as fractions:

In [3]:
(+ 1 1/6)

Fraction(7, 6)

Function definition is straightforward; there is some syntax support in Hy for keyword parameters--more on this below

In [4]:
(defn my-sum [x y]
      (+ x y))

We invoke much like any of the built-ins:

In [5]:
(my_sum 2 3)

5

**Things that can go bump in the night:** Notice that dashes are converted to underscores so `my-sum` shadows `my_sum`. 

The use of `let` allows us to define scopes--really nice. Not sure how to establish that a variable is global (this allows Python functions to modify global variables--I hate this!)

In [6]:
(defn simple-conversation []
      (print "Hello! I'd like to get to know you! Please tell me about yourself!")
      (let [name (raw-input "What is your name? ")
            age (raw-input "What is your age? ")]
        (print (+ "Hello, " name ", I see that you are "
                  age " years old"))))

(simple-conversation)

Hello! I'd like to get to know you! Please tell me about yourself!
What is your name? Brian
What is your age? 42
Hello, Brian, I see that you are 42 years old


The tradition of Lisp lives:

In [7]:
(print (car [1 2 3]))

(print (cdr '(a b c)))

1
('b' 'c')


Notice that Hy supports symbols!!!!!

In [8]:
(print (symbol? 'a))

(symbol? (car '(a b c)))

True


True

What about all the rich Python classes and their methods?

In [9]:
(sort [5 4 3 2 1])

Traceback (most recent call last):
  File "/home/afmoreno/bin/anaconda3/envs/hy-env/lib/python3.6/site-packages/calysto_hy/kernel.py", line 92, in do_execute_direct
    eval(code, self.env)
  File "In [9]", line 1, in <module>
NameError: name 'sort' is not defined



The reason this fails is because `sort` is a method of the Python list `[5 4 3 2 1]`. Thus, we need to invoke the **sort method**

In [10]:
(.sort [5 4 3 2 1])

Of course, if we want to display the results, we need to be explicit about is because the Python list sort method operates in place--one of the really annoying things about Python (though some folks see it as a virtue).

In [11]:
(let [blip [5 4 3 2 1]]
  (.sort blip)
  blip)

[1, 2, 3, 4, 5]

Alternatively, we could have used the `sorted` function

In [12]:
(sorted [5 4 3 2 1])

[1, 2, 3, 4, 5]

# A Lisp-flavored Python

The lines above suggest that Hy is not really a Lisp: side-effects are not something one can avoid easily and still harness all the power of the Python libraries (the reason one might want to play with Hy). The path for dealing with things is mostly "pythonic" (not sure if this is good or bad!). For example, consider looping:

In [13]:
(for [i (range 10)]
  (print (+ "'i' is now at " (str i))))

'i' is now at 0
'i' is now at 1
'i' is now at 2
'i' is now at 3
'i' is now at 4
'i' is now at 5
'i' is now at 6
'i' is now at 7
'i' is now at 8
'i' is now at 9


Using Python's libraries is straighforward and we have a couple of ways we can invoke them:

In [14]:
(import os)

(print (os.getcwd))

(print (.getcwd os))

/home/afmoreno/data/presentations/hy
/home/afmoreno/data/presentations/hy


Or if we want to do more exciting things...

In [15]:
(list-comp x (x (os.listdir)) (!= (first x) "."))

['Hy-tutorial.ipynb', 'Hy-lang.pdf']

Or we could have done something like this...

In [16]:
(list-comp x (x (os.listdir)) (!= (get x 0) "."))

['Hy-tutorial.ipynb', 'Hy-lang.pdf']

The above is the downside of imposing a Lisp syntax on Python: how do we operate on Python lists? These lists are more like vectors, so we need an accessor function, hence `get`.

Well, how about macros? Python doesn't have a REPEAT...UNTIL but, wouldn't it be nice if it did?

In [17]:
(defmacro repeat-until [cond body]
   `(do ~body
      (while (not ~cond) ~body)))

(let [x 0]
  (repeat-until 
     (>= x 5)
     (do (setv x (+ x 1))
         (print x))))

1
2
3
4
5


How does this work? Well, let's see what `macroexpand` has to say about it!

In [18]:
(macroexpand 
     '(repeat-until 
     (>= x 5)
     (do (setv x (+ x 1))
         (print x))))

('do' ('do' ('setv' 'x' ('+' 'x' 1)) ('print' 'x')) ('while' ('not' ('>=' 'x' 5)) ('do' ('setv' 'x' ('+' 'x' 1)) ('print' 'x'))))

** Leveraging Python's Ecosystem **

It is easy  to access Python's ecosystem from within Hy: we can do all the usual imports and even rename them, much like in standard Python.

In the example below we read a file using `Pandas` and then plot it using `Seaborn`.

In [19]:
; %matplotlib notebook: commented out because I can't get this to work. I think
; the Hy kernel for Jupyter has an issue

In [20]:
(import [pandas :as pd])
(import [matplotlib [pyplot :as plt]])
(import [seaborn :as sns])

In [21]:
(setv df (pd.read_csv :filepath_or_buffer "/home/afmoreno/Pokemon.csv"
                      :index_col 0))
; df = pd.read_csv('Pokemon.csv', index_col=0)

In [22]:
(df.head)

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Stage,Legendary
#,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,2,False
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,3,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,2,False


In [23]:
; sns.lmplot(x='Attack', y='Defense', data=df)
(sns.lmplot :x "Attack"
            :y "Defense"
            :data df)
(plt.show)

In [ ]:
!ls

In [ ]:
%%shell
ls